##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# REST API：調整入門指南


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/palm_docs/tuning_quickstart_rest"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/palm_docs/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/palm_docs/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />檢視 GitHub 上的原始程式碼</a>
  </td>
  <td>
    <a target="_blank" href="https://ai.google.dev/palm_docs/tuning_quickstart_rest"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />下載筆記本</a>
  </td>
</table>


在這個筆記本中，你將學習如何使用 curl 命令或 Python request API 開始使用 PaLM API 調校服務來呼叫 PaLM REST API。在此，你將學習如何調校 PaLM API 文本生成服務背後的文本模型。


**注意** ：目前，微調僅提供給`text-bison-001`模型。


## 設定


### 驗證


PaLM API 讓你調整你自己的資料模型。由於這是你的資料以及你調整的模型，這需要比 API 金鑰更嚴格的存取控制。

在你執行這份教學課程前，你需要
[為你的專案設定 OAuth](oauth_quickstart.ipynb)。

如果你想要在 Colab 上執行這份筆記，先使用「檔案 > 上傳」選項上傳你的
`client_secret*.json` 檔案。

![Show colab's File > Upload option](https://developers.generativeai.google/tutorials/images/colab_upload.png)


In [ ]:
!cp client_secret*.json client_secret.json
!ls

client_secret.json

此 gcloud 指令會將 `client_secret.json` 檔案轉成可對服務進行驗證的認證憑據。

重要提示：如果你在 Colab 中執行此操作，**請勿只要按它列印的連結** 。這樣會失敗。請遵照指示，並將它列印的 `gcloud` 指令複製至你的本機電腦上並在該處執行，然後將本機電腦的輸出貼回此處。


In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

## 使用 CURL 呼叫 REST API


本節提供範例的 curl 陳述式以呼叫 REST API。你將學習如何建立調整工作、檢查其狀態並且一完成就發出推論呼叫。


### 設定變數


設定變數供重複值在後續其他 REST API 呼叫時使用。程式碼使用 Python `os` 函式庫設定環境變數，可在所有程式碼單元存取。

這特定於 Colab notebook 環境。下一個程式碼單元中的程式碼等於在 bash 終端機中執行下列命令。

```bash
export access_token=$(gcloud auth application-default print-access-token)
export project_id=my-project-id
export base_url=https://generativelanguage.googleapis.com
```


In [ ]:
import os

access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

os.environ['access_token'] = access_token
os.environ['project_id'] = "project-id"
os.environ['base_url'] = "https://generativelanguage.googleapis.com"


### 列出已調整模型

透過列出目前可用的已調整模型來驗證你的驗證設定。


In [ ]:
%%bash

curl -X GET ${base_url}/v1beta3/tunedModels \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" | grep name


      "name": "tunedModels/testnumbergenerator-fvitocr834l6",
      "name": "tunedModels/my-display-name-81-9wpmc1m920vq",
      "displayName": "my display name 81",
      "name": "tunedModels/number-generator-model-kctlevca1g3q",
      "name": "tunedModels/my-display-name-81-r9wcuda14lyy",
      "displayName": "my display name 81",
      "name": "tunedModels/number-generator-model-w1eabln5adwp",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17583    0 17583    0     0  51600      0 --:--:-- --:--:-- --:--:-- 51563


### 建立調整過的模型

如要建立調整過的模型，你需要在 `training_data` 欄位傳遞資料集給模型。

在此例中，你將調整模型以產生序列中的下一個數字。例如，輸入為  "1"  時，模型應輸出  "2"。輸入為  "一百"  時，輸出應為  "一百零一"。


In [ ]:
%%bash

curl -X POST ${base_url}/v1beta3/tunedModels \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '
      {
        "display_name": "number generator model",
        "base_model": "models/text-bison-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 2,
            "learning_rate": 0.001,
            "epoch_count":3,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    "text_input": "1",
                    "output": "2",
                },{
                    "text_input": "3",
                    "output": "4",
                },{
                    "text_input": "-3",
                    "output": "-2",
                },{
                    "text_input": "twenty two",
                    "output": "twenty three",
                },{
                    "text_input": "two hundred",
                    "output": "two hundred one",
                },{
                    "text_input": "ninety nine",
                    "output": "one hundred",
                },{
                    "text_input": "8",
                    "output": "9",
                },{
                    "text_input": "-98",
                    "output": "-97",
                },{
                    "text_input": "1,000",
                    "output": "1,001",
                },{
                    "text_input": "10,100,000",
                    "output": "10,100,001",
                },{
                    "text_input": "thirteen",
                    "output": "fourteen",
                },{
                    "text_input": "eighty",
                    "output": "eighty one",
                },{
                    "text_input": "one",
                    "output": "two",
                },{
                    "text_input": "three",
                    "output": "four",
                },{
                    "text_input": "seven",
                    "output": "eight",
                }
              ]
            }
          }
        }
      }' | tee tunemodel.json


{
  "name": "tunedModels/number-generator-model-q2d0uism5ivd/operations/xvyx09sjxlmh",
  "metadata": {
    "@type": "type.googleapis.com/google.ai.generativelanguage.v1beta3.CreateTunedModelMetadata",
    "totalSteps": 23,
    "tunedModel": "tunedModels/number-generator-model-q2d0uism5ivd"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2277    0   297  100  1980    146    975  0:00:02  0:00:02 --:--:--  1121


### 取得已調整的模型狀態


在訓練期間，模型狀態設定為 `CREATING`，一旦完成將變更為 `ACTIVE`。

以下是少量的 python 程式碼，用於從回應 JSON 中分析產生的模型名稱。如果你在終端機執行這項工作，你可以嘗試使用 bash JSON parser 來分析回應。


In [ ]:
import json

first_page = json.load(open('tunemodel.json'))
os.environ['modelname'] = first_page['metadata']['tunedModel']

print(os.environ['modelname'])


tunedModels/number-generator-model-q2d0uism5ivd


使用模型名稱執行另一個 `GET` 要求以取得模型中繼資料，其中包含狀態欄位。


In [ ]:
%%bash

curl -X GET ${base_url}/v1beta3/${modelname} \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \ | grep state


  "state": "CREATING",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   494    0   494    0     0    760      0 --:--:-- --:--:-- --:--:--   760
curl: (3) URL using bad/illegal format or missing URL


### 執行推理

一旦你的調整工作完成，你可以用它來使用文字服務產生文字。


In [ ]:
%%bash

curl -X POST ${base_url}/v1beta3/${modelname}:generateText \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '{
        "prompt": {
              "text": "4"
              },
        "temperature": 1.0,
        "candidate_count": 2}' | grep output

      "output": "3 2 1",
      "output": "3 2",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1569    0  1447  100   122    183     15  0:00:08  0:00:07  0:00:01   310


模型的輸出可能正確或不正確。如果調校過的模型未達到你的要求標準，你可以嘗試新增更多高品質範例、調整超參數或為你的範例新增前導文字。你甚至可以根據你建立的第一個範例建立另一個調校過後的模型。

請參閱 [調校指南](../guide/model_tuning_guidance) 以取得更多提高效能的指南。


## 使用 Python requests 呼叫 REST API

你可以使用允許你傳送 http 要求的任何函式庫呼叫 rest API。
下一組範例使用 Python requests 函式庫，並展示更進階功能。


### 設定變數


In [ ]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = 'project-id'
base_url = "https://generativelanguage.googleapis.com"


匯入`requests`函式庫。


In [ ]:
import requests
import json

### 列出已調整模型

透過列出目前可用的已調整模型來驗證你的驗證設定。


In [ ]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta3/tunedModels',
  headers = headers,
)

In [ ]:
result.json()

{'tunedModels': [{'name': 'tunedModels/testnumbergenerator-fvitocr834l6',
   'baseModel': 'models/text-bison-001',
   'displayName': 'test_number_generator',
   'description': '{"description":"generates the  next number in the sequence given the input text","exampleInput":"input: 1","exampleOutput":"output: 2","datasourceUrl":"https://drive.google.com/open?id=11Pdm6GNom4vlBMUHwO6yFjGQT3t1yi44WVShXMFnkVA&authuser=0&resourcekey=0-2d17tccbdBoThXMkNDvtag","showedTuningComplete":false}',
   'state': 'ACTIVE',
   'createTime': '2023-09-18T11:06:39.092786Z',
   'updateTime': '2023-09-18T11:07:24.198359Z',
   'tuningTask': {'startTime': '2023-09-18T11:06:39.461814784Z',
    'completeTime': '2023-09-18T11:07:24.198359Z',
    'snapshots': [{'step': 1,
      'meanLoss': 16.613504,
      'computeTime': '2023-09-18T11:06:44.532937624Z'},
     {'step': 2,
      'epoch': 1,
      'meanLoss': 20.299532,
      'computeTime': '2023-09-18T11:06:47.825134421Z'},
     {'step': 3,
      'epoch': 1,
      'm

### 建立調整過的模型


與 Curl 範例相同，你透過 `training_data` 欄位傳入資料集。


In [ ]:
operation = requests.post(
    url = f'{base_url}/v1beta3/tunedModels',
    headers=headers,
    json= {
        "display_name": "number generator",
        "base_model": "models/text-bison-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 4,
            "learning_rate": 0.001,
            "epoch_count":3,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    'text_input': '1',
                    'output': '2',
                },{
                    'text_input': '3',
                    'output': '4',
                },{
                    'text_input': '-3',
                    'output': '-2',
                },{
                    'text_input': 'twenty two',
                    'output': 'twenty three',
                },{
                    'text_input': 'two hundred',
                    'output': 'two hundred one',
                },{
                    'text_input': 'ninety nine',
                    'output': 'one hundred',
                },{
                    'text_input': '8',
                    'output': '9',
                },{
                    'text_input': '-98',
                    'output': '-97',
                },{
                    'text_input': '1,000',
                    'output': '1,001',
                },{
                    'text_input': '10,100,000',
                    'output': '10,100,001',
                },{
                    'text_input': 'thirteen',
                    'output': 'fourteen',
                },{
                    'text_input': 'eighty',
                    'output': 'eighty one',
                },{
                    'text_input': 'one',
                    'output': 'two',
                },{
                    'text_input': 'three',
                    'output': 'four',
                },{
                    'text_input': 'seven',
                    'output': 'eight',
                }
              ]
            }
          }
        }
      }
)

In [ ]:
operation

<Response [200]>

In [ ]:
operation.json()

{'name': 'tunedModels/number-generator-ncqqnysl74dt/operations/qqlbwzfyzn0k',
 'metadata': {'@type': 'type.googleapis.com/google.ai.generativelanguage.v1beta3.CreateTunedModelMetadata',
  'totalSteps': 12,
  'tunedModel': 'tunedModels/number-generator-ncqqnysl74dt'}}

設定一個變數，其名稱為已調整模型，用於其餘呼叫。


In [ ]:
name=operation.json()["metadata"]["tunedModel"]
name


'tunedModels/number-generator-ncqqnysl74dt'

### 取得已調整的模型狀態


你可以透過檢查狀態欄位，來了解調整工作進度。`CREATING` 表示調整工作仍在進行中，而 `ACTIVE` 表示訓練完成，而且經過調整的模型已準備好使用。


In [ ]:
tuned_model = requests.get(
    url = f'{base_url}/v1beta3/{name}',
    headers=headers,
)

In [ ]:
tuned_model.json()

{'name': 'tunedModels/number-generator-ncqqnysl74dt',
 'baseModel': 'models/text-bison-001',
 'displayName': 'number generator',
 'state': 'CREATING',
 'createTime': '2023-09-19T19:56:25.999303Z',
 'updateTime': '2023-09-19T19:56:25.999303Z',
 'tuningTask': {'startTime': '2023-09-19T19:56:26.297862545Z',
  'hyperparameters': {'epochCount': 3, 'batchSize': 4, 'learningRate': 0.001}},
 'temperature': 0.7,
 'topP': 0.95,
 'topK': 40}

以下程式碼每 5 秒檢查一次狀態欄位，直到它不再處於 `CREATING` 狀態。


In [ ]:
import time
import pprint

op_json = operation.json()
response = op_json.get('response')
error = op_json.get('error')

while response is None and error is None:
    time.sleep(31)

    operation = requests.get(
        url = f'{base_url}/v1/{op_json["name"]}',
        headers=headers,
    )

    op_json = operation.json()
    response = op_json.get('response')
    error = op_json.get('error')

    percent = op_json['metadata'].get('completedPercent')
    if percent is not None:
      print(f"{percent:.2f}% - {op_json['metadata']['snapshots'][-1]}")
      print()

if error is not None:
    raise Exception(error)

21.28% - {'step': 40, 'epoch': 10, 'meanLoss': 2.4871845, 'computeTime': '2023-09-20T00:23:55.255785843Z'}

21.28% - {'step': 40, 'epoch': 10, 'meanLoss': 2.4871845, 'computeTime': '2023-09-20T00:23:55.255785843Z'}

43.09% - {'step': 81, 'epoch': 21, 'meanLoss': 0.032220088, 'computeTime': '2023-09-20T00:24:56.302837803Z'}

43.09% - {'step': 81, 'epoch': 21, 'meanLoss': 0.032220088, 'computeTime': '2023-09-20T00:24:56.302837803Z'}

63.83% - {'step': 120, 'epoch': 32, 'meanLoss': 0.0030430648, 'computeTime': '2023-09-20T00:25:57.228615435Z'}

63.83% - {'step': 120, 'epoch': 32, 'meanLoss': 0.0030430648, 'computeTime': '2023-09-20T00:25:57.228615435Z'}

85.11% - {'step': 160, 'epoch': 42, 'meanLoss': -1.1145603e-06, 'computeTime': '2023-09-20T00:26:57.819011896Z'}

100.00% - {'step': 188, 'epoch': 50, 'meanLoss': 0.00040101097, 'computeTime': '2023-09-20T00:27:40.024132813Z'}



### 執行推論

當微調工作完成後，你可以使用它來和使用基本文字模型相同的方式產生文字。


In [ ]:
import time

m = requests.post(
    url = f'{base_url}/v1beta3/{name}:generateText',
    headers=headers,
    json= {
         "prompt": {
              "text": "9"
              },
    })

In [ ]:
import pprint
print(m.json()['candidates'][0]['output'])

9


你模型的輸出是否正確尚不明確。如果調整後的模型未達到你的所需標準，你可以嘗試添加更多高品質範例、調整超參數或為範例添加前置頁。


## 後續步驟

* 參閱 [使用 Python 的調整快速入門](tuning_quickstart_python)，開始使用調整服務編碼。
* 參閱 [調整指南](../guide/model_tuning_guidance)，深入了解如何針對你的使用案例最佳調整模型。
